# Chương 8: Machine Learning
Chương này yêu cầu bạn thực hiện bài toán sentiment analysis trên corpus sentence polarity dataset v1.0 trong Moview Review Data của tác giả Bo Pang và Lillian Lee. Yêu cầu của bài toán sentiment analysis là phân loại các câu thành positive và negative sentiments.

# 70. Download và tiền xử lý dữ liệu
Sử dụng dữ liệu liên quan đến sentiment polarity của các câu (download tại đây), tạo dữ liệu chuẩn hoá (sentiment.txt) theo hướng dẫn dưới đây.

1. Thêm vào '+1' ở bắt đầu các dòng trong file rt-polarity.pos (giữa +1 và nội dung của câu cách nhau bởi ký tự trắng).

2. Thêm vào '-1' ở bắt đầu các dòng trong file rt-polarity.neg (giữa -1 và nội dung của câu cách nhau bởi ký tự trắng).

3. Kết hợp nội dung thu được trong phần 1 và 2 để tạo thành file sentiment.txt

Sau khi đã thu được file sentiment.txt, xác nhận số lượng các câu với positive polarity và các câu với negative polarity.

In [2]:
# Read files
with open('rt-polaritydata/rt-polaritydata/rt-polarity.pos', 'r') as f:
    pos = f.readlines()
    
with open('rt-polaritydata/rt-polaritydata/rt-polarity.neg', 'r') as f:
    neg = f.readlines()

In [6]:
# Create sentiment.txt
with open('sentiment.txt', 'w') as f:
    for line in pos:
        f.write('+1 ' + line)
    
    for line in neg:
        f.write('-1 ' + line)

In [7]:
# Check sentiment.txt
with open('sentiment.txt', 'r') as f:
    sentiment_data = f.readlines()
    
print(len(sentiment_data))
print('Pos sentence: ' + sentiment_data[0])
print('Neg sentence: ' + sentiment_data[-1])

10662
Pos sentence: +1 the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

Neg sentence: -1 enigma is well-made , but it's just too dry and too placid . 



# 71. Stopwords
Tạo ra danh sách các stopwords trong tiếng Anh. Sau đó viết 1 hàm để kiểm tra một từ có thuộc danh sách stopwords hay không. Hàm sẽ trả về giá trị TRUE nếu từ cho trước thuộc danh sách stopwords. Ngược lại hàm sẽ trả về giá trị FALSE. Sau đó viết mô tả về các test cho hàm đã viết.

In [34]:
def is_stopword(word):
    """
    The function is used to check if a word is stopword
    
    Parameters
    ----------
    word : str
        Word that we want to check, e.g., ``he`` or ``eat``.
    
    Returns
    -------
    out : bool
        True if word in stopwords list.
        False if word not in stopwords list.
        
    Examples
    --------
    >>> is_stopword('eat')
    False
    
    >>> is_stopword('both')
    True
    
    >>> is_stopword(1.5)
    False
    
    >>> s = (2,2)
    >>> is_stopword(s)
    False
    """
    
    # Return False if word is not str
    if not isinstance(word, str):
        return False
    
    stopwords = {"both", "only", "wouldn", "against", "their", "now", "didn", "himself", "ma", "yours", "having", "me", "doesn", "needn", "most", "itself", "m", "s", "isn't", "shan", "did", "won't", "don", "mightn't", "where", "but", "when", "wasn't", "wouldn't", "who", "those", "more", "with", "and", "whom", "an", "into", "before", "you've", "it", "ve", "ain", "haven", "o", "some", "are", "doesn't", "few", "she", "then", "can", "will", "each", "myself", "than", "has", "they", "it's", "there", "hasn", "which", "until", "or", "out", "re", "on", "had", "your", "am", "have", "in", "under", "should", "been", "because", "ourselves", "shouldn't", "that", "too", "the", "from", "didn't", "you'll", "you", "haven't", "don't", "mustn", "hadn", "own", "during", "does", "his", "needn't", "by", "doing", "mustn't", "wasn", "ll", "theirs", "other", "you're", "if", "my", "over", "hasn't", "further", "above", "down", "again", "why", "how", "i", "its", "them", "weren't", "do", "themselves", "between", "through", "here", "weren", "this", "won", "isn", "all", "him", "while", "for", "yourselves", "were", "to", "you'd", "shouldn", "below", "very", "couldn", "about", "she's", "off", "her", "we", "d", "aren't", "just", "what", "yourself", "any", "shan't", "was", "be", "nor", "t", "y", "so", "hadn't", "a", "is", "couldn't", "that'll", "our", "after", "as", "he", "hers", "such", "once", "aren", "these", "herself", "of", "up", "same", "being", "mightn", "ours", "at", "not", "no", "should've"}
    
    return word in stopwords

In [35]:
is_stopword('eat')

False

In [36]:
is_stopword('both')

True

In [37]:
is_stopword(1.5)

False

In [38]:
s = (2,2)
is_stopword(s)

False

# 72. Trích xuất đặc trưng
Tự thiết kế các đặc trưng cho bài toán sentiment analysis. Sau đó trích xuất đặc trưng từ dữ liệu training.

Hint: phương pháp trích xuất đặc trưng đơn giản nhất là sử dụng từ gốc (stem) các từ không trong danh sách các stopwords. Phương pháp này có thể sử dụng để làm hệ thống baseline.

## Read data

In [42]:
import pandas as pd
import random

In [71]:
with open('sentiment.txt', 'r') as f:
    lines = f.readlines()
    
random.shuffle(lines)
    
label = [1 if s.split(' ', 1)[0] == '+1' else 0 for s in lines]
text = [s.strip().split(' ', 1)[1] for s in lines]

data_df = pd.DataFrame(list(zip(text, label)), columns=['comment', 'label'])

data_df.head()

,comment,label
0,"[toback's] fondness for fancy split-screen , s...",0
1,it's nice to see piscopo again after all these...,1
2,a wonderfully warm human drama that remains vi...,1
3,with very little to add beyond the dark vision...,0
4,a derivative collection of horror and sci-fi c...,0


## Simple preprocess

In [72]:
from gensim.utils import simple_preprocess

data_df['simple_preprocessed_comment'] = [simple_preprocess(comment, deacc=True) for comment in data_df['comment']]

In [73]:
data_df.head()

,comment,label,simple_preprocessed_comment
0,"[toback's] fondness for fancy split-screen , s...",0,"[toback, fondness, for, fancy, split, screen, ..."
1,it's nice to see piscopo again after all these...,1,"[it, nice, to, see, piscopo, again, after, all..."
2,a wonderfully warm human drama that remains vi...,1,"[wonderfully, warm, human, drama, that, remain..."
3,with very little to add beyond the dark vision...,0,"[with, very, little, to, add, beyond, the, dar..."
4,a derivative collection of horror and sci-fi c...,0,"[derivative, collection, of, horror, and, sci,..."


## Remove stopword

In [74]:
data_df['rmstopword_comment'] = [[word for word in list_tokens if is_stopword(word) == False] for list_tokens in data_df['simple_preprocessed_comment']]

data_df.head()['rmstopword_comment']

0    [toback, fondness, fancy, split, screen, stutt...
1    [nice, see, piscopo, years, chaykin, headly, p...
2    [wonderfully, warm, human, drama, remains, viv...
3    [little, add, beyond, dark, visions, already, ...
4    [derivative, collection, horror, sci, fi, clic...
Name: rmstopword_comment, dtype: object

## Stemming

In [75]:
from nltk.stem.porter import PorterStemmer
poster = PorterStemmer()

data_df['rmstopword_stemming_comment'] = [[poster.stem(word) for word in list_tokens] for list_tokens in data_df['rmstopword_comment']]

data_df.head()['rmstopword_stemming_comment']

0    [toback, fond, fanci, split, screen, stutter, ...
1    [nice, see, piscopo, year, chaykin, headli, pr...
2    [wonder, warm, human, drama, remain, vividli, ...
3    [littl, add, beyond, dark, vision, alreadi, re...
4             [deriv, collect, horror, sci, fi, clich]
Name: rmstopword_stemming_comment, dtype: object

## Split train test data

In [102]:
from sklearn.model_selection import train_test_split

X = data_df['rmstopword_comment'].apply(lambda x: ' '.join(x))
y = data_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## CountVectorizer

In [110]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(min_df=2, ngram_range=(1, 1))

X_train_count_vect = count_vect.fit(X_train).transform(X_train) 
X_test_count_vect = count_vect.transform(X_test)

In [131]:
# Print score of sentence 0
feature_names = count_vect.get_feature_names()

feature_index = X_train_count_vect[0,:].nonzero()[1]
count_scores = zip(feature_index, [X_train_count_vect[0, x] for x in feature_index])

for w, s in [(feature_names[i], s) for (i, s) in count_scores]:
    print(w, s)

away 1
characters 1
coming 1
everyone 1
except 1
inducing 1
mile 1
see 1
single 1
sleep 1
thriller 1
twist 1
